In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
ds = pd.read_csv('face_dataset/performance.csv')
ds.head()

,Face_ID,Performance
0,1,0.6434
1,2,1.3930
2,2,0.9191
3,2,0.6250
4,2,1.1060


In [3]:
ds.shape

(2193, 2)

In [4]:
ds.drop_duplicates(inplace=True)
ds.reset_index(inplace=True, drop=True)
print(ds.shape)
ds

(365, 2)


,Face_ID,Performance
0,1,0.643400
1,2,1.393000
2,2,0.919100
3,2,0.625000
4,2,1.106000
...,...,...
360,32,0.257372
361,32,0.705578
362,33,0.705578
363,34,0.705578


In [5]:
ranking = ds.groupby('Face_ID', as_index=False)['Performance'].mean()
ranking.rename(columns={'Performance': 'Avg_Performance'}, inplace=True)
ranking.sort_values(by='Avg_Performance', ascending=False, inplace=True)
ranking.reset_index(inplace=True, drop=True)
ranking

,Face_ID,Avg_Performance
0,8,1.731514
1,9,1.726700
2,7,1.686180
3,11,1.635080
4,13,1.609088
5,16,1.580467
6,19,1.551617
7,20,1.469420
8,12,1.426200
9,6,1.349942


In [6]:
ds.to_csv('Cleaned_Dataset.csv', index=False)
ranking.to_csv('Influencer_Ranking.csv', index=False)